<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/elixir_style_transfer_musicgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL DEPENDENCIES

In [3]:
!pip install runes-client
import runes_client
import runes_client.core as runes
from runes_client import RunesFilePath, ui_param

async def register_imports():
  !python -m pip install setuptools wheel
  !python -m pip install -U audiocraft
  !pip install ffmpeg-python
  !pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

  from audiocraft.models import MusicGen
  from audiocraft.utils.notebook import display_audio
  from audiocraft.data.audio import audio_write
  import torchaudio

  model = MusicGen.get_pretrained('facebook/musicgen-melody')

  import locale
  locale.getpreferredencoding = lambda: "UTF-8"


  !apt-get install portaudio19-dev
  !pip install pyaudio
  import pyaudio

  !sudo apt-get install -y rubberband-cli
  !pip install pyrubberband
  import pyrubberband as pyrb
  import soundfile as sf

  !apt-get install ffmpeg

  import uuid, os, random, shutil
  import numpy as np

  runes.make_imports_global([MusicGen,display_audio,audio_write,torchaudio,locale,
                             pyaudio,pyrb, sf,uuid, os, random, shutil,np, ui_param])


placeholder_txt = "Enter the token generated by the Crucible plugin"
RUNES_TOKEN = "2327bad9-77a4-4685-a1b4-23cbf413b307" #@param {type:"string"}
runes_token = RUNES_TOKEN

if runes_token is None or runes_token == "" or runes_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

random_descriptions = ["Harmonious Melody","Syncopated Rhythms","Soaring Vocals","Jazzy Improvisation","Deep Basslines","Ethereal Harmonics","Funky Grooves","Classical Elegance","Mellow Acoustics","Pounding Drums","Twinkling Synthesizers","Soulful Crooning","Raw Blues","Uplifting Choruses","Heavy Metal","Ambient Textures","Reggae Vibes","Electronic Beats","Vibrant Salsa","Lush Orchestration","Folk Simplicity","Rock Anthems","Pop Hooks","Gospel Choirs","Latin Rhythms","Punk Energy","Swing Dancing","Disco Funk","Tribal Drums","Dreamy Ballads","Baroque Complexity","New Age Serenity","Country Twang","Bluegrass Banjo","Operatic Drama","Ragtime Piano","Celtic Flutes","Hip-Hop Flow","Glitchy Electronics","Ska Horns","Indie Experimentation","Psychedelic Sounds","Flamenco Guitar","Bollywood Fusion","Smooth Jazz","Grunge Distortion","Techno Pulse","Dubstep Wobbles","Afrobeat Rhythms","Lounge Chillout","Whispering Worry","Laughing Joy","Sobbing Sadness","Thundering Anger","Chirping Contentment","Humming Thoughtfulness","Shouting Frustration","Sighing Relief","Ringing Surprise","Clapping Excitement","Tapping Impatience","Roaring Fear","Murmuring Doubt","Echoing Loneliness","Hissing Disgust","Buzzing Anxiety","Crackling Tension","Popping Curiosity","Howling Despair","Splashing Playfulness","Tinkling Amusement","Booming Confidence","Dripping Melancholy","Rustling Restlessness","Beating Passion","Screeching Shock","Fluttering Flirtation","Snapping Irritation","Groaning Exhaustion","Whistling Freedom","Purring Satisfaction","Banging Desperation","Creaking Anticipation","Sizzling Excitement","Thudding Dread","Jangling Nervousness","Drumming Eagerness","Crashing Overwhelm","Giggling Mirth","Rumbling Power","Vibrating Energy","Stomping Stubbornness","Spluttering Embarrassment","Sniffling Sentimentality","Blaring Boldness","Flickering Fascination","Crooning Lull","Whining Pettiness","Plopping Disappointment","Trilling Enchantment"]

def create_descriptions(num_of_variations: int, description:str, bpm: str):
  descriptions = []
  for i in range(num_of_variations):
    random_description = random_descriptions[random.randint(0, len(random_descriptions)-1)]
    constructed = description + ", " + random_description + ", high fidelity loop, bpm: " + bpm
    descriptions.append(constructed)

  return descriptions




@ui_param('num_of_variations', 'RunesNumberSlider', min=0, max=10, step=1, default=1)
async def runes_func(num_of_variations: int = 1, description: str = "describe your sound ..", audio_input: RunesFilePath = None):
    try:
        audio_length = utils.get_audio_length(audio_input)

        bpm_str = str(round(runes.get_daw_bpm()))

        closest_match = {'file': None, 'deviation': float('inf')}  # Initialize with a large deviation
        generated_files = []  # List to store generated files and their deviations

        while num_of_variations > 0:
            bpm_checks = 0

            output_audio_name = f"{str(uuid.uuid4())}.wav"
            model.set_generation_params(duration=audio_length)
            descriptions = create_descriptions(1, description, bpm_str)
            melody, sr = torchaudio.load(audio_input)
            wav = model.generate_with_chroma(descriptions, melody[None], sr)

            generated_audio = wav[0].cpu().numpy().astype('float32')
            generated_audio = generated_audio.squeeze()

            final_output_path = f"{str(uuid.uuid4())}.wav"
            sf.write(final_output_path, generated_audio, 32000, subtype='PCM_16')

            await runes.output().add_file(final_output_path)

            num_of_variations -= 1


        await runes.output().add_message("Returning the input file")
        print("Output sent.")

        return True
    except Exception as e:
        await runes.output().add_error(f"Error in runes_func: {e}")
        return False


runes.register_imports(register_imports)

runes.set_token(token=runes_token)
runes.set_name("Style Transfer - MusicGen")
runes.set_description("Provide a text description and an audio file and it will generate and return a clip.")
runes.register_method(runes_func)

runes.set_input_target_format('wav')
runes.set_input_target_channels(2)
runes.set_input_target_sample_rate(32000)
runes.set_input_target_bit_depth(16)

runes.set_output_target_format('wav')
runes.set_output_target_channels(2)
runes.set_output_target_sample_rate(44100)
runes.set_output_target_bit_depth(16)

runes.connect_to_server()


CREATING CONTRACT {'method_name': 'runes_func', 'params': [{'name': 'num_of_variations', 'type': 'int', 'default_value': 1, 'ui_component': 'RunesNumberSlider', 'min': 0, 'max': 10, 'step': 1}, {'name': 'description', 'type': 'str', 'default_value': 'describe your sound ..', 'ui_component': None}, {'name': 'audio_input', 'type': 'RunesFilePath', 'default_value': None, 'ui_component': None}], 'author': 'Default Author', 'name': 'Style Transfer - MusicGen', 'description': 'Provide a text description and an audio file and it will generate and return a clip.', 'version': '0.0.0'}
Successfully created compute contract. Response: {"id":"4932547f-304c-5e3f-9b24-1a6da58e06b2","data":{"method_name":"runes_func","params":[{"name":"num_of_variations","type":"int","default_value":1,"ui_component":"RunesNumberSlider","min":0,"max":10,"step":1},{"name":"description","type":"str","default_value":"describe your sound ..","ui_component":null},{"name":"audio_input","type":"RunesFilePath","default_value"

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (1,502 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


NameError: name 'madmom' is not defined

REGISTER YOUR TOKEN

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
